In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import *
from sklearn.metrics import *

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [ ]:
from tqdm import tqdm

# Load Model

In [ ]:
model = tf.keras.models.load_model("models/model_MMoE_original")

# Define functions

In [ ]:
def get_loader(val_X, num_idx, minp, maxp, maxc, grid_step):
    def gen_p(step=grid_step):
        vx = list(val_X.values[num_idx][5:])
        for p1 in range(minp, maxp + 1, step):
            for p2 in range(minp, min(maxp + 1, maxc + 1 -p1), step):
                for p3 in range(minp, min(maxp + 1, maxc + 1 - p1 - p2), step):
                    for p4 in range(minp, min(maxp + 1, maxc + 1 - p1 - p2 - p3), step):
                        if p1+p2+p3+p4 <= maxc:
                            p = p1+p2+p3+p4
                            yield np.array([p1/p, p2/p, p3/p, p4/p, p/maxc]+vx)
                        
    pred_loader = tf.data.Dataset.from_generator(
                gen_p,
                output_types=tf.dtypes.float64,
                output_shapes=(val_X.shape[1],),
            )

    pred_loader = pred_loader.prefetch(tf.data.experimental.AUTOTUNE).batch(1024)
    
    return pred_loader, list(gen_p())

In [ ]:
def to_perc(x):
    return x.apply(lambda x: x / x.sum(), axis=1)

def process_var(_var, max_len=200):
    var = to_perc(_var)
    var['total_len'] = _var.sum(1) / max_len
    return var

def process_p(p, n_arms=4, max_v=1000):
    for i in range(1, n_arms+1):
        cols = list(filter(lambda x: int(x[0]) == i, p.columns))
        p[f'{i}_v'] = p[cols].sum(1) / max_v
        p[cols] = to_perc(p[cols])
    return p

def process_q(q_, n_arms=4):
    q = q_.copy()
    for i in range(1, n_arms+1):
        q[list(filter(lambda x: len(x) > 1, q.columns[q.columns.str.contains(str(i))]))]  =\
            q[list(filter(lambda x: len(x) > 1, q.columns[q.columns.str.contains(str(i))]))].apply(lambda x: x/q[str(i)])
        q[str(i)] /= 7

    q_ohe = [tf.squeeze(tf.one_hot(feat, 7), 0).numpy() for i, feat in enumerate(q_.values.reshape(-1, 1, 16).T-1)]
    q_ohe = tf.concat(q_ohe, axis=1).numpy()
    q_ohe = pd.DataFrame(q_ohe)
    q = pd.concat([q, q_ohe], axis=1)
    return q

def preprocess_all(p, q, var):
    p = process_p(p)
    q = process_q(q)
    var = process_var(var)
    
    return p, q, var

In [ ]:
def preprocess(df):
    varcols = [f'p{i}' for i in range(1, 5)]
    pcols = df.columns[df.columns.str.contains('_')]
    qcols = ['1', '1l', '1r', '1s', 
             '2', '2l', '2r', '2s', 
             '3', '3l', '3r', '3s',
             '4', '4l', '4r', '4s']
    
    var = df[varcols]
    p = df[pcols]
    q = df[qcols]
    
    p, q, var = preprocess_all(p, q, var)
    
    return df['id'], pd.concat([var, p, q], 1)

In [ ]:
def postprocess_var(var, maxc):
    m = var[4]*maxc
    return [round(x*m, 0) for x in var[:4]]

In [ ]:
def postprocess_y(var):
    var[:,:3] = var[:,:3] * 300
    var[:,3] = var[:,3] * 6 + 3
    var[:,4] = var[:,4] * 3
    
    return var

In [ ]:
performance_measures = ['wait', 'travel', 'timeloss', 'stop', 'speed']

# Predictions on tendency intersection

In [ ]:
df_tendency = pd.read_csv('data/tendency.csv', index_col=0).reset_index(drop=True)

In [ ]:
df_tendency.columns

In [ ]:
strat, X = preprocess(df_tendency)

In [ ]:
test_id = list(tqdm(df_tendency.id.unique()))

In [ ]:
temp_val_X = X[strat.isin(test_id)]
pred = model.predict(temp_val_X) # 0: waiting

In [ ]:
df_sample_test_pred = pd.DataFrame(postprocess_y(pred))

In [ ]:
df_sample_test_pred.columns = ['wait_pred', 'timeloss_pred','travel_pred','speed_pred','stop_pred']
df_sample_test_pred = df_sample_test_pred[[f'{pm}_pred' for pm in performance_measures]]

In [ ]:
df_sample_test_pred

In [ ]:
df_tendency.loc[:, df_sample_test_pred.columns] = df_sample_test_pred

In [ ]:
df_tendency.to_csv('data/tendency_pred.csv')

# Predictions on sample test intersections

In [ ]:
df_sample_test = pd.read_csv(f'data/data_test.csv', index_col=0)

In [ ]:
strat, X = preprocess(df_sample_test)

In [ ]:
test_id = list(tqdm(df_sample_test.id.unique()))

In [ ]:
temp_val_X = X[strat.isin(test_id)]
pred = model.predict(temp_val_X) # 0: waiting

In [ ]:
df_sample_test_pred = pd.DataFrame(postprocess_y(pred))

In [ ]:
df_sample_test_pred.columns = ['wait_pred', 'timeloss_pred','travel_pred','speed_pred','stop_pred']
df_sample_test_pred = df_sample_test_pred[[f'{pm}_pred' for pm in performance_measures]]

In [ ]:
df_sample_test_pred

In [ ]:
df_comparison = df_sample_test

for pm in performance_measures:
    df_comparison.loc[:, pm] = df_comparison[[f'{pm}{i}' for i in range(5)]].mean(axis=1)

In [ ]:
df_comparison = pd.concat([df_comparison, df_sample_test_pred], axis=1)
df_comparison

# Optimization on sample test intersections

In [ ]:
df_sample_test = pd.read_csv(f'data/data_test.csv', index_col=0)

In [ ]:
df_sample_test_for_opt = df_sample_test.drop_duplicates(subset=['id', '1_2', '1_3', '1_4', '2_1', '2_3', '2_4', '3_1', '3_2', '3_4', '4_1', '4_2', '4_3']).reset_index(drop=True)
df_sample_test_for_opt

In [ ]:
strat, X = preprocess(df_sample_test_for_opt)

In [ ]:
test_id = list(tqdm(df_sample_test_for_opt.id.unique()))

### Grid 3

In [ ]:
df_sample_test_for_opt_grid_3 = df_sample_test_for_opt

In [ ]:
min_phase_duration = 5
max_phase_duartion = 120
max_cycle_length = 200 - 12
stepsize = 3

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, stepsize)
    pred = model.predict(pred_loader)[:,0] # 0: waiting
    df_sample_test_for_opt_grid_3.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)])

In [ ]:
df_sample_test_for_opt_grid_3.to_csv('data/optimized_results/opt_grid_3.csv')

### Grid 5

In [ ]:
df_sample_test_for_opt_grid_5 = df_sample_test_for_opt

In [ ]:
min_phase_duration = 5
max_phase_duartion = 120
max_cycle_length = 200 - 12
stepsize = 5

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, stepsize)
    pred = model.predict(pred_loader)[:,0] # 0: waiting
    df_sample_test_for_opt.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)])

In [ ]:
df_sample_test_for_opt_grid_5.to_csv('data/optimized_results/opt_grid_5.csv')

### Grid 7

In [ ]:
df_sample_test_for_opt_grid_7 = df_sample_test_for_opt

In [ ]:
min_phase_duration = 5
max_phase_duartion = 120
max_cycle_length = 200 - 12
stepsize = 7

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, stepsize)
    pred = model.predict(pred_loader)[:,0] # 0: waiting
    df_sample_test_for_opt_grid_7.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)])

In [ ]:
df_sample_test_for_opt_grid_7.to_csv('data/optimized_results/opt_grid_7.csv')

# Optimization on real intersections

In [ ]:
df_real = pd.read_csv(f'data/data_real_intersection.csv', index_col=0)

In [ ]:
df_real_for_opt = df_real.drop_duplicates(subset=['id', '1_2', '1_3', '1_4', '2_1', '2_3', '2_4', '3_1', '3_2', '3_4', '4_1', '4_2', '4_3']).reset_index(drop=True)
df_real_for_opt['idx'] = df_real_for_opt['id']
df_real_for_opt['id'] = 'real'
df_real_for_opt

In [ ]:
strat, X = preprocess(df_real_for_opt)

In [ ]:
test_id = list(tqdm(df_real_for_opt.id.unique()))

#### Loader for real intersection

In [ ]:
def get_loader_real(val_X, num_idx, minp, maxp, maxc, fixed, grid_step):
    def gen_p(step=grid_step):
        vx = list(val_X.values[num_idx][5:])
        for p1 in range(minp[0], maxp[0] + 1, step):
            for p2 in range(minp[1], min(maxp[1] + 1, maxc + 1 -p1), step):
                for p3 in range(minp[2], min(maxp[2] + 1, maxc + 1 - p1 - p2), step):
                    if fixed:
                        if maxp[3] > maxc - p1 - p2 - p3 > minp[3]:
                            p4 = maxc - p1 - p2 - p3
                            p = maxc
                            yield np.array([p1/p, p2/p, p3/p, p4/p, p/maxc]+vx)
                    else:
                        for p4 in range(minp[3], min(maxp[3] + 1, maxc + 1 - p1 - p2 - p3), step):
                            if p1+p2+p3+p4 <= maxc:
                                p = p1+p2+p3+p4
                                yield np.array([p1/p, p2/p, p3/p, p4/p, p/maxc]+vx)
    
    pred_loader = tf.data.Dataset.from_generator(
                gen_p,
                output_types=tf.dtypes.float64,
                output_shapes=(val_X.shape[1],),
            )

    pred_loader = pred_loader.prefetch(tf.data.experimental.AUTOTUNE).batch(1024)
    
    return pred_loader, list(gen_p())

## Original problem

### Grid 3

In [ ]:
df_real_for_opt_grid_3 = df_real_for_opt

In [ ]:
min_phase_duration = [5, 27, 5, 27]
max_phase_duartion = [120, 120, 120, 120]
max_cycle_length = 200 - 12
fixed = False
stepsize = 3

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader_real(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, fixed, stepsize)
    pred = model.predict(pred_loader)[:,0] # 0: waiting
    df_real_for_opt_grid_3.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)],max_cycle_length)

In [ ]:
df_real_for_opt_grid_3.to_csv('data/optimized_results/opt_real_grid_3.csv')

### Grid 5

In [ ]:
df_real_for_opt_grid_5 = df_real_for_opt

In [ ]:
min_phase_duration = [5, 27, 5, 27]
max_phase_duartion = [120, 120, 120, 120]
max_cycle_length = 200 - 12
fixed = False
stepsize = 5

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader_real(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, fixed, stepsize)
    pred = model.predict(pred_loader)[:,0] # 0: waiting
    df_real_for_opt_grid_5.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)],max_cycle_length)

In [ ]:
df_real_for_opt_grid_5.to_csv('data/optimized_results/opt_real_grid_5.csv')

### Grid 7

In [ ]:
df_real_for_opt_grid_7 = df_real_for_opt

In [ ]:
min_phase_duration = [5, 27, 5, 27]
max_phase_duartion = [120, 120, 120, 120]
max_cycle_length = 200 - 12
fixed = False
stepsize = 7

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader_real(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, fixed, stepsize)
    pred = model.predict(pred_loader)[:,0] # 0: waiting
    df_real_for_opt_grid_7.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)],max_cycle_length)

In [ ]:
df_real_for_opt_grid_7.to_csv('data/optimized_results/opt_real_grid_7.csv')

## Fixed Cycle length

### Grid 1

In [ ]:
df_real_for_opt_grid_1 = df_real_for_opt

In [ ]:
min_phase_duration = [5, 27, 5, 27]
max_phase_duartion = [120, 120, 120, 120]
max_cycle_length = 180 - 12
fixed = True
stepsize = 1

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader_real(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, fixed, stepsize)
    pred = model.predict(pred_loader)[:,0] # 0: waiting
    df_real_for_opt_grid_1.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)], max_cycle_length)

In [ ]:
df_real_for_opt_grid_1.to_csv('data/optimized_results/opt_real_fixed_grid_1.csv')

### Grid 3

In [ ]:
df_real_for_opt_grid_3 = df_real_for_opt

In [ ]:
min_phase_duration = [5, 27, 5, 27]
max_phase_duartion = [120, 120, 120, 120]
max_cycle_length = 180 - 12
fixed = True
stepsize = 3

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader_real(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, fixed, stepsize)
    pred = model.predict(pred_loader)[:,0] # 0: waiting
    df_real_for_opt_grid_3.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)],max_cycle_length)

In [ ]:
df_real_for_opt_grid_3.to_csv('data/optimized_results/opt_real_fixed_grid_3.csv')

### Grid 5

In [ ]:
df_real_for_opt_grid_5 = df_real_for_opt

In [ ]:
min_phase_duration = [5, 27, 5, 27]
max_phase_duartion = [120, 120, 120, 120]
max_cycle_length = 180 - 12
fixed = True
stepsize = 5

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader_real(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, fixed, stepsize)
    pred = model.predict(pred_loader)[:,0] # 0: waiting
    df_real_for_opt_grid_5.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)],max_cycle_length)

In [ ]:
df_real_for_opt_grid_5.to_csv('data/optimized_results/opt_real_fixed_grid_5.csv')

### Grid 7

In [ ]:
df_real_for_opt_grid_7 = df_real_for_opt

In [ ]:
min_phase_duration = [5, 27, 5, 27]
max_phase_duartion = [120, 120, 120, 120]
max_cycle_length = 180 - 12
fixed = True
stepsize = 7

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader_real(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, fixed, stepsize)
    pred = model.predict(pred_loader)[:,0] # 0: waiting
    df_real_for_opt_grid_7.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)],max_cycle_length)

In [ ]:
df_real_for_opt_grid_7.to_csv('data/optimized_results/opt_real_fixed_grid_7.csv')

## Multi-objective (travel time 1 speed -10)

### Grid 3

In [ ]:
df_real_for_opt_grid_3 = df_real_for_opt

In [ ]:
min_phase_duration = [5, 27, 5, 27]
max_phase_duartion = [120, 120, 120, 120]
max_cycle_length = 200 - 12
fixed = False
stepsize = 3

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader_real(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, fixed, stepsize)
    pred = model.predict(pred_loader)[:,[2,3]] # 2: travel time, 3: speed
    pred = pred.dot([300, -60]) 
    df_real_for_opt_grid_3.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)],max_cycle_length)

In [ ]:
df_real_for_opt_grid_3.to_csv('data/optimized_results/opt_real_another_pm_grid_3.csv')

### Grid 5

In [ ]:
df_real_for_opt_grid_5 = df_real_for_opt

In [ ]:
min_phase_duration = [5, 27, 5, 27]
max_phase_duartion = [120, 120, 120, 120]
max_cycle_length = 200 - 12
fixed = False
stepsize = 5

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader_real(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, fixed, stepsize)
    pred = model.predict(pred_loader)[:,[2,3]] # 2: travel time, 3: speed
    pred = pred.dot([300, -60]) 
    df_real_for_opt_grid_5.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)],max_cycle_length)

In [ ]:
df_real_for_opt_grid_5.to_csv('data/optimized_results/opt_real_another_pm_grid_5.csv')

### Grid 7

In [ ]:
df_real_for_opt_grid_7 = df_real_for_opt

In [ ]:
min_phase_duration = [5, 27, 5, 27]
max_phase_duartion = [120, 120, 120, 120]
max_cycle_length = 200 - 12
fixed = False
stepsize = 7

In [ ]:
for num_idx, idx in enumerate(X.index):
    pred_loader, pav = get_loader_real(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, fixed, stepsize)
    pred = model.predict(pred_loader)[:,[2,3]] # 2: travel time, 3: speed
    pred = pred.dot([300, -60]) 
    df_real_for_opt_grid_7.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)],max_cycle_length)

In [ ]:
df_real_for_opt_grid_7.to_csv('data/optimized_results/opt_real_another_pm_grid_7.csv')

# Optimization on Individuals of multiple intersections

In [ ]:
df_individual = pd.read_csv(f'data/data_individuals_of_multiple_intersection.csv', index_col=0)

In [ ]:
df_individual_for_opt = df_individual.drop_duplicates(subset=['id', '1_2', '1_3', '1_4', '2_1', '2_3', '2_4', '3_1', '3_2', '3_4', '4_1', '4_2', '4_3']).reset_index(drop=True)
df_individual_for_opt['idx'] = df_individual_for_opt['id']
df_individual_for_opt

In [ ]:
strat, X = preprocess(df_individual_for_opt)

In [ ]:
test_id = list(tqdm(df_individual_for_opt.id.unique()))

#### Loader for real intersection

In [ ]:
def get_loader_real(val_X, num_idx, minp, maxp, maxc, fixed, grid_step):
    def gen_p(step=grid_step):
        vx = list(val_X.values[num_idx][5:])
        for p1 in range(minp[0], maxp[0] + 1, step):
            for p2 in range(minp[1], min(maxp[1] + 1, maxc + 1 -p1), step):
                for p3 in range(minp[2], min(maxp[2] + 1, maxc + 1 - p1 - p2), step):
                    if fixed:
                        if maxp[3] > maxc - p1 - p2 - p3 > minp[3]:
                            p4 = maxc - p1 - p2 - p3
                            p = maxc
                            yield np.array([p1/p, p2/p, p3/p, p4/p, p/maxc]+vx)
                    else:
                        for p4 in range(minp[3], min(maxp[3] + 1, maxc + 1 - p1 - p2 - p3), step):
                            if p1+p2+p3+p4 <= maxc:
                                p = p1+p2+p3+p4
                                yield np.array([p1/p, p2/p, p3/p, p4/p, p/maxc]+vx)
    
    pred_loader = tf.data.Dataset.from_generator(
                gen_p,
                output_types=tf.dtypes.float64,
                output_shapes=(val_X.shape[1],),
            )

    pred_loader = pred_loader.prefetch(tf.data.experimental.AUTOTUNE).batch(1024)
    
    return pred_loader, list(gen_p())

## Fixed Cycle Length (180) & Minimum Durations for Pedestrians

### Grid 5

In [ ]:
df_individual_for_opt_grid_5 = df_individual_for_opt

In [ ]:
min_phase_duration = [5, 5, 5, 5]
max_phase_duartion = [120, 120, 120, 120]
max_cycle_length = 180 - 12
fixed = True
stepsize = 5

In [ ]:
for num_idx, idx in enumerate(X.index):
    if df_individual_for_opt.loc[idx, 'id'] == 'I1':
        min_phase_duration = [5, 43, 5, 30]
    elif df_individual_for_opt.loc[idx, 'id'] == 'I2':
        min_phase_duration = [5, 46, 5, 36]
    elif df_individual_for_opt.loc[idx, 'id'] == 'I3':
        min_phase_duration = [5, 46, 5, 30]
    
    pred_loader, pav = get_loader_real(X, num_idx, min_phase_duration, max_phase_duartion, max_cycle_length, fixed, stepsize)
    pred = model.predict(pred_loader)[:,0] # 0: waiting
    df_individual_for_opt_grid_5.loc[idx, ['p1', 'p2', 'p3', 'p4']] = postprocess_var(pav[np.argmin(pred)],max_cycle_length)

In [ ]:
df_individual_for_opt_grid_5.to_csv('data/optimized_results/opt_individual_of_multi_grid_5.csv')